# Data Wrangling Exercise
Purpose: Classify learners by CEFR

Phase 1: Wrangle some datums

Notes from meeting with Scott:
1. Consider text length
2. Consider how representative each text is (e.g. of a given CEFR band). I am not sure if he was alluding to outliers or something else here.
3. Methods/Technologies to consider:

    a. Semantic spaces

    b. LSA (this was a strong suggestion)

    c. Word2Vec

## Information on EFCAMDAT

> EFCAMDAT consists of essays submitted to Englishtown, the online school of EF Education First, by language learners all over the world (Education First, 2012).  A full course in Englishtown spans 16 proficiency levels aligned with common standards such as TOEFL, IELTS and the Common European Framework of Reference for languages.

__[Overview of EFCAMDAT Data (2013)](https://corpus.mml.cam.ac.uk/faq/SLRF2013Geertzenetal.pdf)__

__[Study with recommendations for Dependency Parsing on this data set (2018)](https://corpus.mml.cam.ac.uk/faq/IJCL2018Huangetal.pdf)__



In [25]:
from lxml import etree
import re # one-way encryption for your codebase
import os.path
import unicodedata # manipulate strange unicode characters
from IPython.display import display # Show me what's going on.
import pandas as pd

print('Working Directory set to:', os.getcwd())

test_file = os.path.join(os.pardir, 'Original Files', 'Level 6 EF_camdat.txt')
with open(test_file, "r") as file:
    data = file.read()

Working Directory set to: /home/jovyan/work/efcamdat/efcamdat-data-cleaning


### One option is to manually alter each illegal character into well-formatted XML. The following script could help with that, but a full-featured text editor like Notepad++ might be a better fit for the job.
__[Predefined characters in XML](https://en.wikipedia.org/wiki/List_of_XML_and_HTML_character_entity_references#Predefined_entities_in_XML)__ are mainly &, ", ', <, and >.

In [26]:
# illegal_character = '&'
# p = re.compile(r'<text>\n+(.+)(' + illegal_character + ')(.+)\s+<\/text>')
# matches = p.findall(data)

# for match in matches[:2]:
#     print(match[1])

### Another option is to wrap every \<TEXT\> block in \[\[CDATA\]\] tags, which might magically make the XML properly formatted
__[CDATA Sections in XML](https://www.tutorialspoint.com/xml/xml_cdata_sections.htm)__

In [27]:
def wrap_cdata(text_masquerading_as_xml):
    return re.sub(r'\s*</text>',r']]></text>', re.sub(r'<text>\s*',r'<text><![CDATA[', text_masquerading_as_xml))

# cdata_blocked_off = wrap_cdata(data)

### This seems to work for some of the levels, but not all of them. Let's try to troubleshoot some more.
Okay. It seems all(?) of the remaining issues are control characters. Python package unicodedata can handle this (thank you
StackOverflow). One way to double check that this is working as expected would be to look at a git difference between
cdata_blocked_off and controls_removed. This way we can ensure that only control characters are being removed.
At first I thought Scott might have intentionally made this data difficult to work with, but this is way
too crazy for anyone to have done it on purpose. One of the responses is literally just a string of control characters??

The remove_control_characters function could be narrowed in scope to just the text within CDATA tags, but I am pretty sure there are no problem bytes anywhere else... therefore it would have the same effect.

In [28]:
def remove_control_characters(not_really_xml):
    return "".join(ch for ch in not_really_xml if unicodedata.category(ch)[0]!="C")

# controls_removed = remove_control_characters(cdata_blocked_off)

In [29]:
def get_writings(cleanish_xml):
    return etree.fromstring(bytes(cleanish_xml, encoding='utf8'))[1]

# parsed_xml_writings = get_writings(controls_removed)
# print("The number of \'samples\' in this level {:,}:".format(len(parsed_xml_writings)))
#some of these samples are obviously useless...

The number of 'samples' in this level 24,789:


### This is looking good! Now we need to build a dataframe and extract the texts.
So we have writing IDs and Learner IDs, so do not make Learner ID the index (indexes should not have duplicate entries).


In [6]:
# Let's just hard code this...

def xml_framer(xml_root, cols):
    lol = [] # list of lists
    for sample in xml_root:
        l = []
        l.append(sample.attrib['id'])
        l.append(sample.attrib['level'])
        l.append(sample.attrib['unit'])
        l.append(sample[0].attrib['id'])
        l.append(sample[0].attrib['nationality'])
        l.append(sample[1].text)
        l.append(sample[1].attrib['id'])
        l.append(sample[2].text)
        l.append(sample[3].text)
        l.append(sample[4].text)
        lol.append(l)
    df = pd.DataFrame(lol, columns=cols)
    return df

col_labels = ['id','lvl','unit','author_id','author_nationality','topic','topic_id','date','grade','text']

# test_df = xml_framer(parsed_xml_writings, col_labels)
# display(test_df)

In [7]:
for i in range(1,17):
    file = os.path.join(os.pardir, 'Original Files', ' '.join(['Level', str(i), 'EF_camdat.txt']))
    print(file)

../Original Files/Level 1 EF_camdat.txt
../Original Files/Level 2 EF_camdat.txt
../Original Files/Level 3 EF_camdat.txt
../Original Files/Level 4 EF_camdat.txt
../Original Files/Level 5 EF_camdat.txt
../Original Files/Level 6 EF_camdat.txt
../Original Files/Level 7 EF_camdat.txt
../Original Files/Level 8 EF_camdat.txt
../Original Files/Level 9 EF_camdat.txt
../Original Files/Level 10 EF_camdat.txt
../Original Files/Level 11 EF_camdat.txt
../Original Files/Level 12 EF_camdat.txt
../Original Files/Level 13 EF_camdat.txt
../Original Files/Level 14 EF_camdat.txt
../Original Files/Level 15 EF_camdat.txt
../Original Files/Level 16 EF_camdat.txt


### Putting it all together
If you have the RAM, might as well...

**Manually fix the mismatched tags (\<user>Theodora Alexopoulou\</url>) in line 8 of Level 6 EF_camdat.txt**

In [30]:
df = pd.DataFrame(columns=col_labels)
for i in range(1,17):
    file = os.path.join(os.pardir, 'Original Files', ' '.join(['Level', str(i), 'EF_camdat.txt']))
    with open(file, "r") as f:
        data = f.read()
        df = df.append(xml_framer(get_writings(remove_control_characters(wrap_cdata(data))), col_labels))
display(df)

,id,lvl,unit,author_id,author_nationality,topic,topic_id,date,grade,text
0,C18217,1,7,20967052,cn,Writing labels for a clothing store,22440,2012-04-20 21:12:14.890,79,Date:monday 11th. Time:9.30 am. From:Margar...
1,C18541,1,1,21016113,cn,Introducing yourself by email,3535,2011-12-24 03:50:49.100,85,"Dear teacher, My name's Yi Zhao,English is poo..."
2,C18648,1,1,20967075,cn,Introducing yourself by email,3535,2012-04-20 08:53:55.087,90,"My name's Henry Hong, I was born in Nanjing o..."
3,C20184,1,1,18898793,cn,Introducing yourself by email,3535,2011-12-14 08:54:48.380,95,"Dear Sir, I'm Jianwen Zhang, from a little tow..."
4,C20185,1,2,18898793,cn,Taking inventory in the office,9820,2011-12-26 09:49:48.140,98,"Dear Ms Thomas, There are thirteen computers a..."
...,...,...,...,...,...,...,...,...,...,...
748,U718945,16,2,21649452,us,Writing about a symbol of your country,8341,2012-09-04 02:47:23.180,65,THE SPACE NEEDLE The United States's Northwest...
749,U719493,16,1,24750663,ae,Attending a robotics conference,7524,2012-09-05 20:41:18.933,0,This is a most new matter of proto and Robots ...
750,U722572,16,5,22117882,br,Using creative writing techniques,9146,2012-09-17 20:59:34.413,96,"Hi Tabby, Never mind about asking me such a ..."
751,U724086,16,2,19328467,sa,Writing about a symbol of your country,8341,2012-09-23 06:55:59.733,94,Oil Tower Statue It is a statue of transitio...


In [31]:
df.to_csv('all_levels.csv')

### We may want to consider running a spellchecker on these responses.

Maybe we...

1. Eliminate useless responses. I think we can assume that all nearly-identical responses have language taken from the prompt. Even if these responses are not just echos of the prompt, I do not think they can tell us very much about the writer, since there is very little variation between them. Like most things, I'm not sure about this.

    a. I think it could be fun to calculate levenshtein distance on the responses, and eliminate responses that are too similar that way.
    
    b. I think the more robust method would be to calculate the TF-IDF and cosine similarity, but that seems a little complex to just find responses with low variance.
    
    c. We could use a cutoff with the "Grade attribute". Irrelevant or incomplete responses are graded lower (maybe anything below 50? below 70?).
    
2. Once we have a relatively useful subset of samples, we can bifurcate

    a. Create .spacy on the samples
    
    b. Run spellchecker then create .spacy on a copy of the samples. This might not buy us anything, but it just might make a difference.
    
    **Actually, I think the best way to do this would be through spaCy.** I don't know if spaCy has an in-built spell checker, but we could add a spell checker to a custom spaCy pipeline. This would have the important advantage of preserving the original, misspelled response as well as a reasonable prediction of the intended orthography. I am not sure how we could incorporate both the uncorrected and corrected texts into a single model, but I sort of like the workflow here anyway.

### Errors I've noticed
1. Spaces before commas. I think this is **always** an error.
2. No space after commas. I think this is an error unless the character after the comma is a quotation mark.
3. Spaces before apostrophes/inverted commas. I think this is **always** an error.
4. Misspellings. I suspect we can improve the data automatically by spell checking, but we of course lose information about writers' spelling knowledge.
5. No space after periods (at end of sentence). This one is tricky. Periods should have a space when they are separating a sentence, but not when they are separating numbers (69.00 dollars) or acronyms (U.S.A.). 